# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-19 21:52:54,857 [INFO] General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
2025-10-19 21:52:54,860 [INFO] ✔ Completed in 0.0007479190826416016 seconds
2025-10-19 21:52:54,864 [INFO] Binding consume in domain (solar, l, q)
2025-10-19 21:52:54,866 [INFO] ✔ Completed in 0.0002560615539550781 seconds
2025-10-19 21:52:54,869 [INFO] General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
2025-10-19 21:52:54,871 [INFO] ✔ Completed in 0.00024890899658203125 seconds
2025-10-19 21:52:54,872 [INFO] Binding consume in domain (wind, l, y)
2025-10-19 21:52:54,874 [INFO] ✔ Completed in 0.00016188621520996094 seconds
2025-10-19 21:52:54,876 [INFO] General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
2025-10-19 21:52:54,882 [INFO] ✔ Completed in 0.0005528926849365234 seconds
2025-10-19 21:52:54,884 [INFO] Binding release in domain (power, l, q)
2025-10-19 21:52:5

## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-19 21:52:55,910 [INFO] Binding capacity in domain (wf, l, y)
2025-10-19 21:52:55,913 [INFO] ✔ Completed in 0.0007181167602539062 seconds
2025-10-19 21:52:55,915 [INFO] Binding capacity in domain (wf, l, y)
2025-10-19 21:52:55,919 [INFO] ✔ Completed in 0.000743865966796875 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-19 21:52:59,690 [INFO] Binding operate in domain (wf, l, q)
2025-10-19 21:52:59,697 [INFO] ✔ Completed in 0.0007641315460205078 seconds
2025-10-19 21:52:59,701 [INFO] Mapping operate: (wf, l, q) → (wf, l, y)
2025-10-19 21:52:59,703 [INFO] ✔ Completed in 0.000s


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-19 21:53:02,352 [INFO] Binding capacity in domain (pv, l, y)
2025-10-19 21:53:02,367 [INFO] ✔ Completed in 0.00039696693420410156 seconds
2025-10-19 21:53:02,383 [INFO] Binding capacity in domain (pv, l, y)
2025-10-19 21:53:02,390 [INFO] ✔ Completed in 0.0003838539123535156 seconds
2025-10-19 21:53:02,404 [INFO] Binding operate in domain (pv, l, q)
2025-10-19 21:53:02,415 [INFO] ✔ Completed in 0.0048601627349853516 seconds
2025-10-19 21:53:02,421 [INFO] Mapping operate: (pv, l, q) → (pv, l, y)
2025-10-19 21:53:02,425 [INFO] ✔ Completed in 0.000s


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-19 21:53:04,770 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-19 21:53:04,782 [INFO] ✔ Completed in 0.00044918060302734375 seconds
2025-10-19 21:53:04,784 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-19 21:53:04,787 [INFO] ✔ Completed in 0.0003719329833984375 seconds
2025-10-19 21:53:04,799 [INFO] General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory(power.lii, l, y)
2025-10-19 21:53:04,806 [INFO] Binding capacity in domain (lii.charge, l, y)
2025-10-19 21:53:04,808 [INFO] ✔ Completed in 0.0002300739288330078 seconds
2025-10-19 21:53:04,809 [INFO] Binding capacity in domain (lii.discharge, l, y)
2025-10-19 21:53:04,811 [INFO] ✔ Completed in 0.00012803077697753906 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-19 21:53:16,718 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-19 21:53:16,722 [INFO] ✔ Completed in 0.0017230510711669922 seconds
2025-10-19 21:53:16,726 [INFO] General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-19 21:53:16,728 [INFO] ✔ Completed in 0.0003991127014160156 seconds
2025-10-19 21:53:16,738 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-19 21:53:16,742 [INFO] ✔ Completed in 0.0005710124969482422 seconds
2025-10-19 21:53:16,745 [INFO] General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-19 21:53:16,750 [INFO] ✔ Completed in 0.00027370452880859375 seconds
2025-10-19 21:53:16,756 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-19 21:53:16,763 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, y)
2025-10-19 21:53:16,765 [INFO]

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [9]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
m.sol()

'Use .opt() to generate solution'

## Optimize!

In [11]:
m.show()

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
m.usd.spend.opt()

2025-10-19 21:53:25,606 [INFO] Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
2025-10-19 21:53:25,636 [INFO] ✔ Completed in 0.000s
2025-10-19 21:53:25,651 [INFO] Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
2025-10-19 21:53:25,653 [INFO] ✔ Completed in 0.001s
2025-10-19 21:53:25,656 [INFO] Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
2025-10-19 21:53:25,658 [INFO] ✔ Completed in 0.000s
2025-10-19 21:53:25,669 [INFO] Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
2025-10-19 21:53:25,681 [INFO] ✔ Completed in 0.000s
2025-10-19 21:53:25,685 [INFO] Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
2025-10-19 21:53:25,686 [INFO] ✔ Completed in 0.000s
2025-10-19 21:53:25,696 [INFO] Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
2025-10-19 21:53:25,698 [INFO] ✔ Completed in 0.000s


Generating Program(example2).mps
Creating gurobi model for Program(example2)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros
Optimizing Program(example2) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 

## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [ ]:
m.inventory.sol()

The amount charged into inventory is:

In [ ]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

The amount discharged from inventory is:

In [ ]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

### Integer Decisions 

All the operations are setup in this case

In [ ]:
m.capacity.reporting.sol()

In [ ]:
m.capacity.sol()